# Chuẩn bị dữ liệu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp drive/MyDrive/contentbase.zip .

Hoặc có thể lấy dữ liệu tại link: https://drive.google.com/file/d/1kMsRqYI8F810_wVASkaSd2yF2iUSdVaL/view?usp=sharing

# Chuẩn bị thư viện

In [ ]:
!pip install -q faiss-cpu sentence-transformers pandas numpy

In [ ]:
!unzip contentbase.zip

Archive:  contentbase.zip
replace contentbase/vector-embedding/faissid_to_movieid.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: ya
  inflating: contentbase/vector-embedding/faissid_to_movieid.pkl  
replace contentbase/vector-embedding/faiss_content.index? [y]es, [n]o, [A]ll, [N]one, [r]ename: Ay
  inflating: contentbase/vector-embedding/faiss_content.index  
  inflating: contentbase/vector-embedding/faiss_people.index  
  inflating: contentbase/vector-embedding/faiss_title.index  
  inflating: contentbase/vector-embedding/faiss_type.index  
  inflating: contentbase/vector-embedding/movie_content.csv  
  inflating: contentbase/vector-embedding/movie_embedding.pkl  
  inflating: contentbase/data/stg_cast.csv  
  inflating: contentbase/data/stg_genre.csv  
  inflating: contentbase/data/stg_keyword.csv  
  inflating: contentbase/data/stg_links.csv  
  inflating: contentbase/data/stg_movie_metadata.csv  


In [ ]:
import faiss
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from typing import Dict, List, Optional

In [ ]:
BASE_PATH = "/content/contentbase"

# Load faiss index và movies data

In [ ]:
DATA_PATH = f"{BASE_PATH}/data"
VEC_PATH  = f"{BASE_PATH}/vector-embedding"

In [ ]:
faiss_indices = {
    "title": faiss.read_index(f"{VEC_PATH}/faiss_title.index"),
    "content": faiss.read_index(f"{VEC_PATH}/faiss_content.index"),
    "type": faiss.read_index(f"{VEC_PATH}/faiss_type.index"),
    "people": faiss.read_index(f"{VEC_PATH}/faiss_people.index"),
}

with open(f"{VEC_PATH}/faissid_to_movieid.pkl", "rb") as f:
    faiss_to_movie = pickle.load(f)

In [ ]:
# load data
movies   = pd.read_csv(f"{DATA_PATH}/stg_movie_metadata.csv")
genres   = pd.read_csv(f"{DATA_PATH}/stg_genre.csv")
cast     = pd.read_csv(f"{DATA_PATH}/stg_cast.csv")
links    = pd.read_csv(f"{DATA_PATH}/stg_links.csv")


genre_agg = (
    genres.groupby("movie_id")["genre"]
    .apply(list)
    .reset_index()
    .rename(columns={"movie_id": "id", "genre": "genres"})
)


cast["person"] = cast["role"] + ":" + cast["name"]
cast_agg = (
    cast.groupby("movie_id")["person"]
    .apply(list)
    .reset_index()
    .rename(columns={"movie_id": "id", "person": "cast"})
)


movies = (
    movies
    .merge(genre_agg,   on="id", how="left")
    .merge(cast_agg,    on="id", how="left")
    .merge(
        links.rename(columns={"movie_id": "id"}),
        on="id",
        how="left"
    )
)


for col in ["genres", "cast"]:
    movies[col] = movies[col].apply(lambda x: x if isinstance(x, list) else [])

movies["overview"] = movies["overview"].fillna("")
movies["tagline"]  = movies["tagline"].fillna("")
movies["title"]    = movies["title"].fillna("")


In [ ]:
movies

,id,title,original_title,belongs_to_collection,original_language,release_date,status,overview,tagline,adult,...,revenue,vote_average,vote_count,imdb_id_x,tmdb_id_x,keywords,genres,cast,imdb_id_y,tmdb_id_y
0,176,Saw,Saw,Saw Collection,en,2004-10-01,Released,Obsessed with teaching his victims the value o...,Live or die. Make your choice.,False,...,103911669,7.2,2255,113677.0,9071.0,"shotgun,based on short film,sadist,pistol,chai...","[Horror, Mystery, Crime]","[cast:Cary Elwes, cast:Danny Glover, cast:Moni...",113677.0,9071.0
1,333318,Scherzo Diabolico,Scherzo Diabolico,NaN,es,2015-04-16,Released,Aram is a wearied accountant with an unbearabl...,,False,...,0,6.0,8,NaN,NaN,kidnapping,"[Horror, Comedy]","[cast:Francisco Barreiro, cast:Daniela Soto Ve...",NaN,NaN
2,82663,Midnight Man,Midnight Man,NaN,en,NaN,NaN,British soldiers force a recently captured IRA...,,False,...,0,1.0,10,NaN,NaN,"undercover,soldier",[],"[cast:Rob Lowe, cast:Kenneth Cranham, cast:Deb...",NaN,NaN
3,122662,マルドゥック・スクランブル 排気,マルドゥック・スクランブル 排気,Mardock Scramble Collection,ja,NaN,NaN,Third film of the Mardock Scramble series.,,False,...,0,1.0,10,NaN,NaN,"artificial intelligence,based on novel,cyberpu...",[],"[cast:Megumi Hayashibara, cast:Hiroki Touchi, ...",NaN,NaN
4,362185,Operator,Operator,NaN,en,2015-08-21,Released,When the daughter of veteran 911 call center o...,Whats your emergency?,False,...,0,5.2,21,NaN,NaN,"hostage,kidnapping,officer,chaos,daughter,poli...","[Thriller, Action, Drama]","[cast:Mischa Barton, cast:Luke Goss, cast:Ving...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45428,36214,Rebirth of Mothra,モスラ,Mothra Collection,ja,1996-12-14,Released,A new Mothra takes to the air and must battle ...,,False,...,0,6.0,7,NaN,NaN,"mothra,kaiju,ghidora","[Adventure, Science Fiction, Action, Fantasy]","[cast:Megumi Kobayashi, cast:Sayaka Yamaguchi,...",NaN,NaN
45429,9898,The Killing Floor,The Killing Floor,NaN,en,2007-05-10,Released,A literary agent moves into a penthouse apartm...,,False,...,0,5.6,14,NaN,NaN,"videoband,neighbor,surveillance","[Crime, Thriller]","[cast:Marc Blucas, cast:Shiri Appleby, cast:Re...",NaN,NaN
45430,88067,Cover-Up,Cover-Up,NaN,en,1991-02-22,Released,"Mike Anderson, a tough American reporter on a ...",If the lies don't kill you – the truth will.,False,...,0,4.3,6,1810633.0,60897.0,"spy,israel,cia,journalist,ambassador,tel aviv,...","[Action, Thriller]","[cast:Dolph Lundgren, cast:Lisa Berkley, direc...",1810633.0,60897.0
45431,71772,Go West Young Man,Go West Young Man,NaN,en,1936-11-18,Released,Mavis Arden (Mae West) is a sensational movie ...,,False,...,0,4.8,5,NaN,NaN,"innuendo,ribald humor",[Comedy],"[cast:Mae West, cast:Warren William, cast:Rand...",NaN,NaN


# Load embedding của phim

Cấu trúc của movie_embeddings:



```
movie_embeddings = {
  movie_id: {
    title_vector: (384,0)
    content_vector: (384, 0)
    type_vector: (384,)
    keyword_vector (384,)
  }
}
```




In [ ]:
with open(f"{VEC_PATH}/movie_embedding.pkl", "rb") as f:
    movie_embeddings = pickle.load(f)

sample_id = next(iter(movie_embeddings))
movie_embeddings[sample_id].keys()

dict_keys(['title_vector', 'content_vector', 'type_vector', 'people_vector'])

# Hàm search FAISS

In [ ]:
VECTOR_DIM = next(iter(movie_embeddings.values()))["title_vector"].shape[0]

def search_by_faiss_index(
    movie_id: int,
    movie_embeddings: dict,
    faiss_indices: dict,
    faiss_to_movie: dict,
    k: int = 10,
):
    vectors = movie_embeddings[movie_id]
    results = []

    for index_name, index in faiss_indices.items():

        if index_name == "title":
            q = vectors["title_vector"]
        elif index_name == "content":
            q = vectors["content_vector"]
        elif index_name == "type":
            q = vectors["type_vector"]
        elif index_name == "people":
            q = vectors["people_vector"]
        else:
            continue

        q = q.reshape(1, -1).astype("float32")
        distances, indices = index.search(q, k)

        for d, idx in zip(distances[0], indices[0]):
            results.append({
                "type": index_name,
                "distance": float(d),
                "movieId": faiss_to_movie[idx]
            })

    return results

# Hàm gộp điểm tương đồng

In [ ]:
WEIGHT = {
    "title": 0.45,
    "content": 0.20,
    "type": 0.10,
    "people": 0.25
}

def aggregate_scores(results, top_k=10):
    scores = {}

    for r in results:
        mid = r["movieId"]
        scores[mid] = scores.get(mid, 0) + r["distance"] * WEIGHT[r["type"]]

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]


# Hàm show kết quả
nhận vào:
- list [(movies_id, score)]
- top_k=10

In [ ]:
def show_results(recs, movies, top_k=10):
    for i, (mid, score) in enumerate(recs[:top_k], 1):
        row = movies.loc[movies.id == mid].iloc[0]

        title = row["title"]
        year = row["release_date"][:4] if pd.notna(row["release_date"]) else "N/A"
        rating = row["vote_average"]
        votes = row["vote_count"]

        genres = ", ".join(row["genres"][:5]) if row["genres"] else "N/A"
        keywords = row["keywords"]
        cast = ", ".join(row["cast"][:5]) if row["cast"] else "N/A"

        overview = row["overview"][:200] + "..." if row["overview"] else ""

        print(f"{i}. {title} ({year})")
        print(f"   ⭐ score={score:.4f} | rating={rating} ({votes} votes)")
        print(f"   🎭 Genres   : {genres}")
        print(f"   🧩 Keywords : {keywords}")
        print(f"   👥 People   : {cast}")
        print(f"   📖 Overview : {overview}")
        print("-" * 80)

# Search by keyword

### *embedding function*

In [ ]:
EMBEDDING_DIM = 384
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embedding(
    text: Optional[str],
    model: SentenceTransformer,
    dim: int = EMBEDDING_DIM
) -> np.ndarray:

    if text is None or not isinstance(text, str) or not text.strip():
        return np.zeros(dim)

    emb = model.encode([text], normalize_embeddings=True)[0]

    if len(emb) != dim:
        raise ValueError("Embedding dimension mismatch")

    return emb

## *search faiss function*

In [ ]:
def multi_search_faiss_index(
    query_emb: np.ndarray,
    index_dict: Dict[str, faiss.Index],
    faiss_to_movie: Dict[int, int],
    k: int = 10,
) -> List[dict]:

    results = []
    query_emb = query_emb.reshape(1, -1)

    for index_name, index in index_dict.items():
        distances, indices = index.search(query_emb, k)

        for dist, idx in zip(distances[0], indices[0]):
            if idx == -1:
                continue

            results.append({
                "type": index_name,
                "distance": float(dist),
                "faiss_id": int(idx),
                "movieId": int(faiss_to_movie[idx])
            })

    return results

## *search function*

In [ ]:
def search_movies(
    query: str,
    movies: pd.DataFrame,
    faiss_indices: Dict[str, faiss.Index],
    faiss_to_movie: Dict[int, int],
    top_k: int = 10,
):
    query = query.strip()
    if len(query) < 3:
        raise ValueError("Query must be at least 3 characters")

    # embedding
    query_embedding = get_embedding(query, embedding_model)

    # FAISS search
    raw_results = multi_search_faiss_index(
        query_embedding,
        faiss_indices,
        faiss_to_movie,
        k=top_k
    )

    movie_scores = {}
    for r in raw_results:
        movie_id = r["movieId"]
        score = r["distance"]
        movie_scores[movie_id] = max(movie_scores.get(movie_id, 0), score)

    # sort
    sorted_movies = sorted(
        movie_scores.items(),
        key=lambda x: x[1],
        reverse=True
    )[:top_k]

    return sorted_movies


In [ ]:
res = search_movies("batman", movies, faiss_indices, faiss_to_movie)

In [ ]:
show_results(res, movies)

1. Batman (1966)
   ⭐ score=0.9454 | rating=6.1 (209 votes)
   🎭 Genres   : Crime, Family, Adventure, Comedy, Science Fiction
   🧩 Keywords : submarine,dc comics,missile,shark attack,rescue,shark,shark repelent,black cat,super powers
   👥 People   : cast:Adam West, cast:Burt Ward, cast:Lee Meriwether
   📖 Overview : The Dynamic Duo faces four super-villains who plan to hold the world for ransom with the help of a secret invention that instantly dehydrates people....
--------------------------------------------------------------------------------
2. Batman: Gotham Knight (2008)
   ⭐ score=0.8539 | rating=6.7 (171 votes)
   🎭 Genres   : Animation, Adventure, Action
   🧩 Keywords : dc comics,anthology,super powers
   👥 People   : cast:Kevin Conroy, cast:Gary Dourdan, cast:Jason Marsden, director:Yasuhiro Aoki
   📖 Overview : Explore Bruce Wayne's transition from his beginning as a tormented vigilantee to The Dark Knight of a crumbling metropolis with six distinct chapters but intended to 

# Content-base recommendation

In [ ]:
movie_id = movies["id"].iloc[0]

In [ ]:
show_results([(movie_id, 1)], movies)

1. Saw (2004)
   ⭐ score=1.0000 | rating=7.2 (2255 votes)
   🎭 Genres   : Horror, Mystery, Crime
   🧩 Keywords : shotgun,based on short film,sadist,pistol,chained,bludgeoning,game of death
   👥 People   : cast:Cary Elwes, cast:Danny Glover, cast:Monica Potter, director:James Wan
   📖 Overview : Obsessed with teaching his victims the value of life, a deranged, sadistic serial killer abducts the morally wayward. Once captured, they must face impossible choices in a horrific game of survival. T...
--------------------------------------------------------------------------------


In [ ]:

raw_results = search_by_faiss_index(
    movie_id,
    movie_embeddings,
    faiss_indices,
    faiss_to_movie,
    k=10
)

recommendations = aggregate_scores(raw_results, top_k=5)

show_results(recommendations, movies)


1. Saw (2004)
   ⭐ score=1.0000 | rating=7.2 (2255 votes)
   🎭 Genres   : Horror, Mystery, Crime
   🧩 Keywords : shotgun,based on short film,sadist,pistol,chained,bludgeoning,game of death
   👥 People   : cast:Cary Elwes, cast:Danny Glover, cast:Monica Potter, director:James Wan
   📖 Overview : Obsessed with teaching his victims the value of life, a deranged, sadistic serial killer abducts the morally wayward. Once captured, they must face impossible choices in a horrific game of survival. T...
--------------------------------------------------------------------------------
2. Saw V (2008)
   ⭐ score=0.4078 | rating=6.0 (848 votes)
   🎭 Genres   : Horror, Thriller, Crime
   🧩 Keywords : blood splatter,serial killer,pig mask,nazi,jigsaw,swastika tattoo
   👥 People   : cast:Tobin Bell, cast:Costas Mandylor, cast:Scott Patterson, director:David Hackl
   📖 Overview : Detective Hoffman is seemingly the last person alive to carry on the Jigsaw legacy. But when his secret is threatened, he mu

In [ ]:
# ===== Build ground truth từ collection =====

movies["collection_name"] = (
    movies["belongs_to_collection"]
    .astype(str)
    .str.strip()
)

movies.loc[
    movies["collection_name"].isin(["", "nan", "None"]),
    "collection_name"
] = np.nan

collection_groups = (
    movies
    .dropna(subset=["collection_name"])
    .groupby("collection_name")["id"]
    .apply(list)
)

gt = {}
for cname, mids in collection_groups.items():
    if len(mids) <= 1:
        continue
    for mid in mids:
        gt[mid] = [m for m in mids if m != mid]

len(gt)


4098

In [ ]:
def recommend_movie(mid, top_k=10):
    if mid not in movie_embeddings:
        return []

    raw_results = search_by_faiss_index(
        movie_id=mid,
        movie_embeddings=movie_embeddings,
        faiss_indices=faiss_indices,
        faiss_to_movie=faiss_to_movie,
        k=top_k
    )

    ranked = aggregate_scores(raw_results, top_k=top_k)

    # bỏ chính nó
    return [m for m, _ in ranked if m != mid]

In [ ]:
def precision_recall_at_k(recommended, ground_truth, k=10):
    recommended = recommended[:k]
    gt_set = set(ground_truth)

    hit = len(set(recommended) & gt_set)

    precision = hit / k
    recall = hit / len(gt_set) if len(gt_set) > 0 else 0.0

    return precision, recall


In [ ]:
precisions = []
recalls = []

for mid, gt_items in gt.items():
    recs = recommend_movie(mid, top_k=10)

    if len(recs) == 0:
        continue

    p, r = precision_recall_at_k(recs, gt_items, k=10)
    precisions.append(p)
    recalls.append(r)

len(precisions), np.mean(precisions), np.mean(recalls)


(4098, np.float64(0.30200097608589554), np.float64(0.939851913881158))